# Credit Risk Resampling Techniques

In [1]:
import warnings


warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from collections import Counter
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import ClusterCentroids
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
import pickle







# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('forlex.csv')
df = pd.read_csv(file_path)
df.head()

,ID,LIMIT_BAL,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_pay,Total_Missed_Pymts,Payment_Ratio,Total_Bill_AMT,Total_Payment,%_Amount_Paid,Risk_Level
0,1,20000.0,1,24,1,1,0,0,0,0,...,0.0,0.0,0.0,1,2,33.333333,7704.0,689.0,8.943406,Moderate
1,2,120000.0,2,26,0,1,0,0,0,1,...,1000.0,0.0,2000.0,1,2,33.333333,17077.0,5000.0,29.279147,Moderate
2,3,90000.0,2,34,0,0,0,0,0,0,...,1000.0,1000.0,5000.0,0,0,0.000000,101653.0,11018.0,10.838834,NaN
3,4,50000.0,1,37,0,0,0,0,0,0,...,1100.0,1069.0,1000.0,0,0,0.000000,231334.0,8388.0,3.625926,NaN
4,5,50000.0,1,57,0,0,0,0,0,0,...,9000.0,689.0,679.0,0,0,0.000000,109339.0,59049.0,54.005433,NaN


In [4]:
X=df[['MARRIAGE', 'AGE','PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6','Total_Missed_Pymts', 'Payment_Ratio','Total_Payment']]


In [5]:
y=df[['default_pay']]

# Split the Data into Training and Testing

In [6]:
# Check the balance of our target values

y['default_pay'].value_counts()

0    23364
1     6636
Name: default_pay, dtype: int64

In [7]:
# Create X_train, X_test, y_train, y_test
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [8]:
print(f'''% Positive class in Train ={np.round(y_train.value_counts(normalize=True)[1] *100, 2)}''')

% Positive class in Train =default_pay
1              22.08
dtype: float64


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [9]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
# YOUR CODE HERE
scaler = StandardScaler()



In [10]:
 #Fit the Standard Scaler with the training data
 #When fitting scaling functions, only train on the training dataset
#YOUR CODE HERE
X_scaler = scaler.fit(X_train)


In [11]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [12]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

ros = RandomOverSampler(random_state=1)
# resampling X, y
X_ros, y_ros = ros.fit_resample(X_train_scaled, y_train)
#ros_preds=ros.predict(X_test)
# new class distribution 
print(f''' Shape of X before ROS: { X_train_scaled.shape}
Shape of X after ROS: {X_ros.shape}''')

# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_ros.value_counts(normalize=True) * 100)

 Shape of X before ROS: (22500, 11)
Shape of X after ROS: (35064, 11)


Counter({50.0: 2})

In [13]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
ros_model= LogisticRegression(solver='lbfgs', random_state=1)
ros_model.fit(X_ros, y_ros)

LogisticRegression(random_state=1)

In [14]:
pickle.dump(ros_model,open('naive_ros.pkl', 'wb'))


In [15]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
ros_y_pred = ros_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, ros_y_pred)

0.6950143588832638

In [17]:
print(f"Training Data Score: {ros_model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {ros_model.score(X_test_scaled, y_test)}")

Training Data Score: 0.7718666666666667
Testing Data Score: 0.7706666666666667


In [18]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, ros_y_pred)

array([[4848,  984],
       [ 736,  932]], dtype=int64)

In [19]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, ros_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.87      0.83      0.56      0.85      0.68      0.48      5832
          1       0.49      0.56      0.83      0.52      0.68      0.45      1668

avg / total       0.78      0.77      0.62      0.78      0.68      0.47      7500

